# HERON Paths

In [40]:
import os
import sys
import operator
import numpy as np
import pickle
import subprocess
import shutil
import pandas as pd
import time as time_mod
import matplotlib.pyplot as plt
import matplotlib.ticker as mt
import seaborn as sns

sys.path.append('..')

from ExternalDispatchManager import ExternalDispatchRunner

raven = "~/projects/raven/raven_framework"
heron = "~/projects/HERON/heron"
mra   = "~/projects/multiresolutionanalysis"

RAVEN_LOC = os.path.expanduser(raven)
RAVEN_DIR = os.path.abspath( os.path.join(RAVEN_LOC, '..') )
sys.path.append(RAVEN_LOC)
sys.path.append(RAVEN_DIR)

HERON_LOC = os.path.expanduser(heron)
HERON_DIR = os.path.abspath( os.path.join(HERON_LOC, '..') )
sys.path.append(HERON_LOC)
sys.path.append(HERON_DIR)

MRA_LOC = os.path.expanduser(mra)
sys.path.append(os.path.join(MRA_LOC, '..'))
sys.path.append(MRA_LOC)

from src.DispatchManager import DispatchRunner
from ravenframework import ROMExternal
from ravenframework.utils import xmlUtils

In [10]:
from HERON.src.dispatch.MRA_testing.utils.process_files import (get_heron_lib,
                                                                grab_sources,
                                                                populate_raven_dict)

from multiresolutionanalysis.notebooks.dispatch.utils.parsing_results import (fix_path,
                                                                              check_dir,
                                                                              check_and_make_dir)

# Loop through Cases

In [11]:
scratch_dir = check_dir( '/scratch/sotogj/MRADispatch/NYISO/dispatch' )
CASES = [
    'Case1',
    'Case2a','Case2b',
    'Case3a','Case3b','Case3c',
    'Case4a','Case4b','Case4c',
    'Case5a', 'Case5d',
]

In [51]:
CASE_INFO = {}
CASE_INFO['ROM']     = 'fGOLD'
CASE_INFO['METRIC']  = 'NPV_C'
CASE_INFO['SAMPLES'] = 1

CASE_OUTPUT = {}
CASE_OUTPUT['total_time'] = {}
CASE_OUTPUT['sum_time']   = {}
CASE_OUTPUT['metrics']    = {}

for CASE in CASES:

    CASE_OUTPUT['total_time'][CASE] = {}
    # define case
    CASE_INFO['CASE']  = CASE

    # define input and output locations for file transfer
    MRA_scripts_dir = check_dir( os.path.join(mra, 'output/DISPATCH_COLLECT') )
    MRA_case_dir    = check_dir( os.path.join(scratch_dir, CASE_INFO['ROM'], CASE) )
    MRA_test_dir    = check_dir( os.path.join(MRA_case_dir, 'dispatchOpt') )
    if not MRA_test_dir:
        break

    # run HERON and get HERON lib file location
    removeCapex = True

    # collect all info from the run and load it from a pickle file
    filename = f"{CASE}_{CASE_INFO['ROM']}_"
    filename += 'dNC_' if removeCapex else 'd_'
    filename += f"{CASE_INFO['METRIC']}"
    # current_time = time_mod.strftime("%Y_%m_%d", time_mod.localtime())
    current_time = '2024_09_25'
    collection_file = fix_path( os.path.join(MRA_test_dir,
                                            f'collect_{filename}__{current_time}.pk') )

    # collection['raven_dict'] = raven_dict
    # collection['case_info']  = CASE_INFO
    # collection['dispatch_runner'] = dispatch_runner
    # collection['dispatch'] = dispatch
    # collection['metrics'] = metrics
    # collection['tot_activity'] = tot_activity
    # collection['time_elapsed'] = end_time-start_time

    with open(collection_file, 'rb') as fp:
        collection = pickle.load(fp)

    sum_time = []
    for macro,macroInfo in collection['dispatch'].items():
        for cluster,clusterInfo in macroInfo.items():
            sum_time.append(clusterInfo.get_elapsed_time())

    CASE_OUTPUT['total_time'][CASE] = collection['time_elapsed']
    CASE_OUTPUT['sum_time'][CASE]   = sum_time
    CASE_OUTPUT['metrics'][CASE]    = collection['metrics']

In [75]:
fig = plt.figure(figsize=(14,8), facecolor='white')
gs = fig.add_gridspec(3,1)

axM = fig.add_subplot(gs[0,0])
axD = fig.add_subplot(gs[1,0])
axN = fig.add_subplot(gs[2,0])


NPVs   = [val['NPV']*1e-6 for val in CASE_OUTPUT['metrics'].values()]
totalTimes = [val for val in CASE_OUTPUT['total_time'].values()]
sumTimes   = [np.sum(val) for val in CASE_OUTPUT['sum_time'].values()]
caseNames  = [val for val in CASE_OUTPUT['total_time']]

distTimes  = [val for val in CASE_OUTPUT['sum_time'].values()]

xArray = range(len(caseNames))
axM.semilogy(totalTimes, 'v', label='Full Time')
axM.semilogy(sumTimes, '^', label='Sum of All Time Windows')
for i,(t,s) in enumerate(zip(totalTimes,sumTimes)):
    axM.text(xArray[i]+0.075, totalTimes[i],    f"{t:.2f}s", fontsize=8)
    axM.text(xArray[i]+0.075, sumTimes[i]*0.85, f"{100*(t-s)/t:.2f}%", fontsize=8)
# X-axis
axM.set_xticks(xArray)
axM.set_xticklabels(caseNames)
axM.set_xlim(-0.5,len(xArray)+0.5)
# Y-axis
axM.set_yticks( [50, 100, 200, 500, 1000, 2000])
axM.get_yaxis().set_major_formatter(mt.ScalarFormatter())
axM.get_yaxis().set_tick_params(which='minor', size=0)
axM.get_yaxis().set_tick_params(which='minor', width=0)
axM.set_ylim(10**(1.5),10**(3.5))
axM.set_ylabel('Elapsed Time \nLog-Scale (s)')
axM.grid(True)
axM.legend()

sns.violinplot(data=distTimes, ax=axD, log_scale=True)
# axD.violinplot(distTimes, range(len(caseNames)))
# X-axis
axD.set_xticks(range(len(caseNames)))
axD.set_xticklabels(caseNames)
axD.set_xlim(-0.5,len(xArray)+0.5)
# Y-axis
axD.set_ylabel('Elapsed Time \nPer Time Window (1-yr) \nLog-Scale (s)')
axD.set_yticks( [1,3,5,10,20,50])
axD.get_yaxis().set_major_formatter(mt.ScalarFormatter())
axD.get_yaxis().set_tick_params(which='minor', size=0)
axD.get_yaxis().set_tick_params(which='minor', width=0)
axD.grid(True)
# axD.set_ylim(10**(1.5),10**(3.5))

axN.bar(xArray, NPVs, color='green')
axN.set_xticks(range(len(caseNames)))
axN.set_xticklabels(caseNames)
axN.set_xlim(-0.5,len(xArray)+0.5)
axN.set_ylim(80,120)
axN.set_ylabel('NPV \nw/o CAPEX ($M)')

fig.suptitle('Single Inner, Project Time=30yr')
plt.tight_layout()
plt.savefig('ElapsedTimes', bbox_inches='tight')